In [ ]:
import numpy as np
import pandas as pd
import skyfield
from skyfield.api import Loader
skyfield_load = Loader('../data/skyfield')
from skyfield.positionlib import ICRF, Barycentric
from skyfield.toposlib import Topos
import astropy
from astropy.units import deg, au, km, meter, day, minute, second
from astropy.coordinates import SkyCoord, ICRS, GCRS, BarycentricMeanEcliptic, HeliocentricMeanEcliptic, EarthLocation
from scipy.interpolate import CubicSpline
import os
import matplotlib.pyplot as plt

# MSE imports
import astro_utils
from astro_utils import jd_to_mjd, mjd_to_jd
from ra_dec import radec2dir, dir2radec, site2geoloc, qv2dir, direction_diff, radec_diff, skyfield_observe
from ra_dec import load_pos_jpl, load_ast_jpl, load_obs_jpl, load_obs_ast_jpl
from ra_dec import obs_add_interp_qv, obs_ast_add_interp_qv, obs_add_calc_dir, obs_add_radec, obs_direction_diff
import asteroid_integrate
from asteroid_data import make_data_one_file, get_earth_pos
from utils import range_inc

### Observation of Earth and Mars according to JPL

In [ ]:
# Data directories
dir_name = '../data/jpl/testing/hourly'
dir_name_daily = '../data/jpl/testing/daily'

# Build DataFrame for earth and mars position at 3 hour frequency
df_earth = load_pos_jpl(body_name='earth', dir_name=dir_name)
df_mars = load_pos_jpl(body_name='mars', dir_name=dir_name)

# Earth at daily frequency
df_earth_daily = load_pos_jpl(body_name='earth', dir_name=dir_name_daily)
# df_mars_daily = load_pos_jpl(body_name='mars', dir_name=dir_name_daily)

In [ ]:
# Display the earth dataframe
df_earth

In [ ]:
# Display the mars dataframe
# df_mars

In [ ]:
# Load the JPL observations of Mars
df_obs_mars_geo = load_obs_jpl(body_name='mars', observer_name='geocenter', dir_name=dir_name)
df_obs_mars_pal = load_obs_jpl(body_name='mars', observer_name='palomar', dir_name=dir_name)

# Display the dataframe
df_obs_mars_geo

In [ ]:
# Extract position and velocity of earth from df_earth
q_earth_jpl = np.array([df_earth.X.values, df_earth.Y.values, df_earth.Z.values]) * au
v_earth_jpl = np.array([df_earth.VX.values, df_earth.VY.values, df_earth.VZ.values]) * au / day

# Extract position of mars from df_mars
q_mars_jpl = np.array([df_mars.X.values, df_mars.Y.values, df_mars.Z.values]) * au
v_mars_jpl = np.array([df_mars.VX.values, df_mars.VY.values, df_mars.VZ.values]) * au / day

# Extract obstime_jd, ra, and dec from DataFrame with geocentric observations
obstime_mars_geo_jd = df_obs_mars_geo.JulianDate.values
# ra_mars_geo_jpl = df_obs_mars_geo.RA.values * deg
# dec_mars_geo_jpl = df_obs_mars_geo.DEC.values * deg

# Observation times for palomar observations
obstime_mars_pal_jd = df_obs_mars_pal.JulianDate.values

# Vector of observation times in MJD format
obstime_mars_geo_mjd = jd_to_mjd(obstime_mars_geo_jd)
obstime_mars_pal_mjd = jd_to_mjd(obstime_mars_pal_jd)

# Alias to obstime_mars_mjd because they are the same
obstime_mars_jd = obstime_mars_geo_jd
obstime_mars_mjd = obstime_mars_geo_mjd

### Position & Observation of Earth and Mars according to Skyfield

In [ ]:
# Manually load planetary positions using de435
# planets_sf = skyfield_load('../data/jpl/ephemeris/de435.bsp')
planets_sf = skyfield_load('de435.bsp')
earth_sf = planets_sf['earth']
mars_sf = planets_sf['mars barycenter']

# load timescale
ts_sf = skyfield_load.timescale()

# Generate vector of observation times in Skyfield format
obstime_mars_sf = ts_sf.tt_jd(obstime_mars_jd)

In [ ]:
# Observe mars from earth geocenter with Skyfield
ra_mars_geo_sf, dec_mars_geo_sf, delta_mars_geo_sf = \
    skyfield_observe(observer_sf=earth_sf, body_sf=mars_sf, obstime_sf=obstime_mars_sf)

In [ ]:
# location of palomar as a Skyfield topos object
geoloc_pal = site2geoloc('palomar', verbose=True)
lon, lat, height = geoloc_pal.geodetic
palomar_topos = skyfield.toposlib.Topos(latitude_degrees=lat.value, longitude_degrees=lon.value, elevation_m=height.value)
palomar_topos

In [ ]:
# Observe mars from palomar with Skyfield
palomar_sf = earth_sf + palomar_topos
ra_mars_pal_sf, dec_mars_pal_sf, delta_mars_pal_sf = \
    skyfield_observe(observer_sf=palomar_sf, body_sf=mars_sf, obstime_sf=obstime_mars_sf)

In [ ]:
# Load planetary positions and velocities by querying the Skyfield JPL ephemeris interface
# Create them as arrays with bundled astropy units of au and km / second

# Earth
q_earth_sf = earth_sf.at(obstime_mars_sf).ecliptic_position().au * au
v_earth_sf = earth_sf.at(obstime_mars_sf).ecliptic_velocity().km_per_s * km / second

# Palomar
q_palomar_sf = palomar_sf.at(obstime_mars_sf).ecliptic_position().au * au
v_palomar_sf = palomar_sf.at(obstime_mars_sf).ecliptic_velocity().km_per_s * km / second

# Mars
q_mars_sf = mars_sf.at(obstime_mars_sf).ecliptic_position().au * au
v_mars_sf = mars_sf.at(obstime_mars_sf).ecliptic_velocity().km_per_s * km / second

In [ ]:
# Demonstrate that q_earth_sf is the same as q_earth_jpl
q_earth_eps = np.mean(np.linalg.norm(q_earth_sf - q_earth_jpl, axis=0))
v_earth_eps = np.mean(np.linalg.norm(v_earth_sf - v_earth_jpl, axis=0))
q_mars_eps = np.mean(np.linalg.norm(q_mars_sf - q_mars_jpl, axis=0))
v_mars_eps = np.mean(np.linalg.norm(v_mars_sf - v_mars_jpl, axis=0))

# Report
print('Difference between Skyfield (JPL ephem) and Horizons download:')
print(f'q_earth : {q_earth_eps:5.3e} au')
print(f'v_earth : {v_earth_eps:5.3e} au / day')
print(f'q_mars  : {q_mars_eps:5.3e} au')
print(f'v_mars  : {v_mars_eps:5.3e} au / day')

**Conclusion**<br>
Skyfield is essentially identical to JPL in coordinates of Earth and Mars.
Difference is on the order of $10^{-9}$ AU.<br>

### Compare Skyfield vs JPL on RA/DEC of Mars

In [ ]:
# Convert SkyField RA/DEC to directions
# u_mars_geo_sf = radec2dir(ra=ra_mars_geo_sf, dec=dec_mars_geo_sf, obstime_mjd=obstime_mars_mjd)
# u_mars_pal_sf = radec2dir(ra=ra_mars_pal_sf, dec=dec_mars_pal_sf, obstime_mjd=obstime_mars_mjd)

In [ ]:
# Add RA/DEC and direction from Skyfield to mars observation frames
obs_add_radec(df_obs=df_obs_mars_geo, ra=ra_mars_geo_sf, dec=dec_mars_geo_sf, source='sf')
obs_add_radec(df_obs=df_obs_mars_pal, ra=ra_mars_pal_sf, dec=dec_mars_pal_sf, source='sf')

In [ ]:
# Report difference between JPL and Skyfield from geocenter
print(f'Comparing direction of Mars from Geocenter: Skyfield vs. JPL')
print(f'(1) Direction according to JPL: radec2dir applied to JPL RA/DEC')
print(f'(2) Direction according to Skyfield: radec2dir applied to Skyfield RA/DEC (from observe)\n')
diff_geo_jpl_sf = obs_direction_diff(df_obs=df_obs_mars_geo, src1='jpl', src2='sf', verbose=True)

In [ ]:
# Report difference between JPL and Skyfield from palomar
print(f'Comparing direction of Mars from Palomar: Skyfield vs. JPL')
print(f'(1) Direction according to JPL: radec2dir applied to JPL RA/DEC')
print(f'(2) Direction according to Skyfield: radec2dir applied to Skyfield RA/DEC (from observe)\n')
diff_geo_pal_sf = obs_direction_diff(df_obs=df_obs_mars_pal, src1='jpl', src2='sf', verbose=True)

In [ ]:
# Review observation frame with additional columns
df_obs_mars_geo

**Conclusion**<br>
Skyfield is very close to JPL  on observation of Mars.<br>
Mean difference is **1.60 arc seconds** from both Geocenter and Palomar.<br>
The difference between Geocenter and Palomar is about 3.78 arc seconds on average for Mars.<br>
Results appear to be consistent between Geocenter and Palomar.

### Estimate Importance of Including Observatory Location

In [ ]:
# Difference between palomar and geocenter according to JPL
radec_diff(name1='geocenter-JPL', name2='palomar-JPL', 
           ra1=df_obs_mars_geo.RA_jpl.values*deg, dec1=df_obs_mars_geo.DEC_jpl.values*deg,
           ra2=df_obs_mars_pal.RA_jpl.values*deg, dec2=df_obs_mars_pal.DEC_jpl.values*deg,
           obstime_mjd=df_obs_mars_geo.mjd.values, verbose=True)

In [ ]:
# Difference between palomar and geocenter according to Skyfield
radec_diff(name1='geocenter-Skyfield', name2='palomar-Skyfield', 
           ra1=df_obs_mars_geo.RA_sf.values*deg, dec1=df_obs_mars_geo.DEC_sf.values*deg,
           ra2=df_obs_mars_pal.RA_sf.values*deg, dec2=df_obs_mars_pal.DEC_sf.values*deg,
           obstime_mjd=df_obs_mars_geo.mjd.values, verbose=True)

**Conclusion**<br>
Ignoring the observatory location would introduce an error of about **3.8 arc seconds** for Mars.<br>
This effect is important enough that we should certainly try to model it.

### Calculate Direction from Earth to Mars with qv2dir() and JPL Position / Velocity

In [ ]:
# Add interpolated JPL Positions to observation DataFrames
obs_add_interp_qv(df_obs=df_obs_mars_geo, df_body=df_mars, df_earth=df_earth, source_name='jpl')
obs_add_interp_qv(df_obs=df_obs_mars_pal, df_body=df_mars, df_earth=df_earth, source_name='jpl')

In [ ]:
# Display augmented df_obs_mars
df_obs_mars_geo

In [ ]:
# Build geolocation of theoretical observer at geocenter
geoloc_geo = site2geoloc(site_name='geocenter', verbose=False)

# Build geolocation of observer at Palomar
geoloc_pal = site2geoloc(site_name='palomar', verbose=True)

In [ ]:
# Compute the directions qv2dir() accounting for observer location; save them to the DataFrame of mars observations
obs_add_calc_dir(df_obs=df_obs_mars_geo, site_name='geocenter', source_name='jpl')
obs_add_calc_dir(df_obs=df_obs_mars_pal, site_name='palomar', source_name='jpl')

In [ ]:
# Review added columns
df_obs_mars_geo

In [ ]:
# Review all the columns that are now on the observation DataFrame
df_obs_mars_geo.columns

### Direction from Geocenter to Mars: Compare JPL, Skyfield, and qv2dir(JPL position)

In [ ]:
# Report difference for Mars from Geocenter between Skyfiled and MSE calculated
print(f'Comparing direction of Mars from Geocenter: Skyfield vs. MSE calc from JPL positions:')
print(f'(1) Direction according to Skyfield: radec2dir applied to Skyfield RA/DEC')
print(f'(2) Direction according to MSE: qv2dir applied to JPL positions & velocities\n')
diff_geo_sf_calc_jpl = obs_direction_diff(df_obs=df_obs_mars_geo, src1='sf', src2='calc_jpl', verbose=True)

In [ ]:
# Report difference for Mars from Geocenter between JPL and MSE calculated
print(f'Comparing direction of Mars from Geocenter: JPL vs. MSE calc from JPL positions')
print(f'(1) Direction according to JPL: radec2dir applied to JPL RA/DEC')
print(f'(2) Direction according to MSE: qv2dir applied to JPL positions & velocities\n')
diff_geo_jpl_jpl_calc_jpl = obs_direction_diff(df_obs=df_obs_mars_geo, src1='jpl', src2='calc_jpl', verbose=True)

**Conclusion**<br>
My calculations are almost identical to Skyfield; accurate to **0.027 arc seconds**<br>
Both Skyfield and I are off from JPL by **1.60 arc seconds**<br>

### Direction from Palomar to Mars: Compare JPL, Skyfield, and qv2dir(JPL position)

In [ ]:
# Calculate direction from palomar using JPL positions
obs_add_calc_dir(df_obs=df_obs_mars_pal, site_name='palomar', source_name='jpl')

In [ ]:
# Report difference for Mars from Palomar between Skyfield and MSE calculated
print(f'Comparing direction of Mars from Palomar: Skyfield vs. MSE calc from JPL positions:')
print(f'(1) Direction according to Skyfield: radec2dir applied to Skyfield RA/DEC')
print(f'(2) Direction according to MSE: qv2dir applied to JPL positions & velocities\n')
diff_pal_sf_calc_jpl = obs_direction_diff(df_obs=df_obs_mars_pal, src1='sf', src2='calc_jpl', verbose=True)

In [ ]:
# Report difference for Mars from Geocenter between JPL and MSE calculated
print(f'Comparing direction of Mars from Geocenter: JPL vs. MSE calc from JPL positions')
print(f'(1) Direction according to JPL: radec2dir applied to JPL RA/DEC')
print(f'(2) Direction according to MSE: qv2dir applied to JPL positions & velocities\n')
diff_pal_jpl_calc_jpl = obs_direction_diff(df_obs=df_obs_mars_pal, src1='jpl', src2='calc_jpl', verbose=True)

### Vectors of First 16 Asteroids from JPL

In [ ]:
# Load the asteroid position and velocity from JPL
df_ast = load_ast_jpl(ast_num0=1, ast_num1=16, dir_name=dir_name)
df_ast_daily = load_ast_jpl(ast_num0=1, ast_num1=16, dir_name=dir_name_daily)

In [ ]:
df_ast

### Observations of First 16 Asteroids from JPL

In [ ]:
# Load the asteroid observations from JPL
df_obs = load_obs_ast_jpl(ast_num0=1, ast_num1=16, observer_name='palomar', dir_name=dir_name)
df_obs_daily = load_obs_ast_jpl(ast_num0=1, ast_num1=16, observer_name='geocenter', dir_name=dir_name_daily)

In [ ]:
df_obs

### Calculate Asteroid Direction with qv2dir() and JPL Position / Velocity

In [ ]:
# Add interpolated JPL position to asteroid observations
obs_ast_add_interp_qv(df_obs=df_obs, df_ast=df_ast, df_earth=df_earth, source_name='jpl')

In [ ]:
# Add computed directions from the JPL positions
obs_add_calc_dir(df_obs=df_obs, site_name='palomar', source_name='jpl')

In [ ]:
# Review asteroid observations with additional columns: interpolated positions & calculated RA/DEC, direction
df_obs

In [ ]:
# Report difference for between JPL and MSE calculated
print(f'Comparing direction of Asteroids from Palomar: JPL vs. MSE calc from JPL positions')
print(f'(1) Direction according to JPL: radec2dir applied to JPL RA/DEC')
print(f'(2) Direction according to MSE: qv2dir applied to JPL positions & velocities\n')
diff_ast_jpl_calc_jpl = obs_direction_diff(df_obs=df_obs, src1='jpl', src2='calc_jpl', verbose=True)

**Conclusion:<br>**
qv2dir() is highly accurate in computing a right ascension and declination from position and velocity in the barycentric ecliptic plane.<br>
Errors are on the order of **0.87 arc seconds**.<br>
Differences with JPL are due to using a linear approximation to the adjustment of the space body's position due to light lag.  The JPL calculation is iteratively solving for the position of the body on its true orbit at the instant photons leaving it hit the earth at print time.  This simplified calculation is applying an adjustment of the form<br>
```
r = norm(q_body - q_earth)
light_time = r / light_speed
dq = v_body * light_time
```

In [ ]:
# Check that round trip between RA/DEC and direction is accurate

# extract u_jpl from df_obs
u_jpl = df_obs[['ux_jpl', 'uy_jpl', 'uz_jpl']].values
# extract RA, DEC and obstime_mjd
ra_jpl = df_obs.RA_jpl.values * deg
dec_jpl = df_obs.DEC_jpl.values * deg
obstime_mjd = df_obs.mjd.values

# Compute RA and DEC from direction computed by radec2dir() on the JPL data
ra_jpl2, dec_jpl2 = dir2radec(u_jpl, obstime_mjd)

In [ ]:
# Compute difference in angles
diff_rt = radec_diff('JPL', 'MSE', ra1=ra_jpl, dec1=dec_jpl, ra2=ra_jpl2, dec2=dec_jpl2, 
                     obstime_mjd=obstime_mjd, verbose=True)
print(f'\nMean difference on round trip = {diff_rt:5.3e} arc seconds.')

**Conclusion:<br>**
The round trip of radec2dir() and dir2radec() is accurate on the order of double precision.<br>
In the test, a direction was computed from the RA and DEC provided by JPL. This was then converted back to a RA and DEC.<br>
Errors are on the order of **5.8E-11 arc seconds**.<br>

### Calculate Asteroid Direction with qv2dir() and MSE Position / Velocity

In [ ]:
ast_elt = asteroid_integrate.load_data()
ast_elt.rename(mapper={'Num':'asteroid_num'}, axis='columns', inplace=True)

In [ ]:
# Range of asteroids to for data
ast_num_file_start: int = 1
ast_num_file_end: int = 1000
inputs, outputs = make_data_one_file(0, ast_num_file_end)

In [ ]:
ast_elt

In [ ]:
# The block of asteroid numbers to test (inclusive boundaries)
ast_num_min = 1
ast_num_max = 16

# The number of asteroids, times, and total rows we want to match
# Use daily data to match data file
N_ast = ast_num_max - ast_num_min + 1
N_t = df_earth_daily.mjd.size
N_row = N_ast * N_t
obstime_mjd = df_ast_daily.mjd.values

# Report data shape
print(f'Shape of data frames df_ast and df_obs:')
print(f'N_ast = {N_ast:5} asteroids')
print(f'N_t   = {N_t:5} observation times')
print(f'N_row = {N_row:5} rows in df_ast and df_obs')

In [ ]:
# Filter for asteroid numbers 
ast_num_file = np.arange(ast_num_file_start, ast_num_file_end, dtype=np.int64)
mask_ast = (ast_num_min <= ast_num_file) & (ast_num_file <= ast_num_max)

# MSE integrated times as one array
ts = inputs['ts'][0]

# Time range for JPL data
t_min = np.min(obstime_mjd)
t_max = np.max(obstime_mjd)

# Filter for MSE times that match
mask_t = (t_min <= ts) & (ts <= t_max)

In [ ]:
# Block of asteroid data 
q_ast_all = outputs['q']
v_ast_all = outputs['v']

# filter for selected asteroids only
q_ast_all_t = q_ast_all[mask_ast, :, :]
v_ast_all_t = v_ast_all[mask_ast, :, :]

# filter for selected times only
q_ast_mse_3d = q_ast_all_t[:, mask_t, :]
v_ast_mse_3d = v_ast_all_t[:, mask_t, :]

# for some reason i don't understand, can't do these at once
# q_ast_mse = q_ast_all[mask_ast, mask_t, :]
q_ast_mse_3d.shape

In [ ]:
# Get position of Earth using utility function
q_earth_mse_tile = get_earth_pos(obstime_mjd).transpose()

q_earth_mse_tile.shape

In [ ]:
# shape JPL positions to match q_ast_mse with three axes (asteroid_num, time_idx, space_dim)
q_ast_jpl_3d = np.zeros((N_ast, N_t, 3))
q_ast_jpl_3d[:, :, 0] = df_ast_daily.X.values.reshape((N_ast, N_t))
q_ast_jpl_3d[:, :, 1] = df_ast_daily.Y.values.reshape((N_ast, N_t))
q_ast_jpl_3d[:, :, 2] = df_ast_daily.Z.values.reshape((N_ast, N_t))
q_ast_jpl_3d.shape

In [ ]:
# Reshape MSE asteroid data to match shape of DataFrame
q_ast_mse = np.zeros((3, N_row))
v_ast_mse = np.zeros((3, N_row))

In [ ]:
# Position
q_ast_mse[0, :] = q_ast_mse_3d[:, :, 0].reshape((-1))
q_ast_mse[1, :] = q_ast_mse_3d[:, :, 1].reshape((-1))
q_ast_mse[2, :] = q_ast_mse_3d[:, :, 2].reshape((-1))

# Velocity
v_ast_mse[0, :] = v_ast_mse_3d[:, :, 0].reshape((-1))
v_ast_mse[1, :] = v_ast_mse_3d[:, :, 1].reshape((-1))
v_ast_mse[2, :] = v_ast_mse_3d[:, :, 2].reshape((-1))

In [ ]:
df_obs_daily

In [ ]:
# Compute direction from MSE position and velocity
u_mse = qv2dir(q_body=q_ast_mse*au, v_body=v_ast_mse*au/day, q_earth=q_earth_mse_tile*au)

# extract u_jpl from df_obs_daily
u_jpl = df_obs_daily[['ux_jpl', 'uy_jpl', 'uz_jpl']].values.T

# difference in directions as a vector
u_diff = u_mse - u_jpl

# norm of difference, converted to arc seconds
u_diff_norm = np.linalg.norm(u_diff, axis=0)
angle_diff = np.rad2deg(u_diff_norm)*3600

# mean error in arc-seconds
mean_error = np.mean(angle_diff)
print(f'mean error: {mean_error:8.3f} arc seconds')

**Conclusion:<br>**
My end to end calculation of astromentric RA and DEC are very close to those of JPL.<br>
In my calculation, I am only taking a single snapshot of planetary positions and velocities, plus orbital elements of the asteroids.  Everything else is done by numerically integrating the system in rebound.
I am computing an astrometric direction u on the unit sphere in the ecliptic frame, and comparing this to a direction from JPL.  The JPL direction u_jpl is computed by applying radec2dir() on the quoted RA and DEC.<br>
Errors are on the order of **3.6 arc seconds**.<br>
I am guessing that one main source for the difference with JPL is that I used heliocentric rather than barycentric coordinates when saving the outputs of the rebound integration.  I plan to switch to barycentric for the asteroid search.  Of course there are also some other differences because these are completely separate calculations.  JPL in particular is using many more massive bodies, and they are accounting for relativistic effects.<br>
Still, the bottom line is that an agreement of only 3.6 arc seconds is very tight and suggests that my methodology is basically sound.

### Astrometric vs. Apparent Coordinates

In [ ]:
df_obs

**JPL Definitions of Astrometric & Apparent RA/DEC**

 R.A._______(ICRF)_______DEC =
  Astrometric right ascension and declination of the target center with
respect to the observing site (coordinate origin) in the reference frame of
the planetary ephemeris (ICRF). Compensated for down-leg light-time delay
aberration.

  Units: RA  in decimal degrees (ddd.fffffffff)
         DEC in decimal degrees (sdd.fffffffff)

 
 R.A._______(airless-appar)_______DEC. =
  Airless apparent right ascension and declination of the target center with
respect to an instantaneous reference frame defined by the Earth equator
of-date (z-axis) and meridian containing the Earth equinox of-date (x-axis,
IAU76/80). Compensated for down-leg light-time delay, gravitational deflection
of light, stellar aberration, precession & nutation. Note: equinox (RA origin)
is offset -53 mas from the of-date frame defined by the IAU06/00a P & N system.

  Units: RA  in decimal degrees (ddd.fffffffff)
         DEC in decimal degrees (sdd.fffffffff)

In [ ]:
# alias the astrometric RA/DEC so the variable names look consistent
# these are the astrometric RA/DEC
ra_astro = ra_jpl
dec_astro = dec_jpl

# arrays of apparent asteroid angles from JPL
ra_appar = df_obs.RA_apparent.values * deg
dec_appar = df_obs.DEC_apparent.values * deg

# Compute difference in angles
diff_app = radec_diff('Astrometric', 'Apparent', ra1=ra_astro, dec1=dec_astro, ra2=ra_appar, dec2=dec_appar, 
                     obstime_mjd=obstime_mjd, verbose=False)
diff_mean = np.mean(diff_app)
diff_median = np.median(diff_app)
diff_max = np.max(diff_app)

# Report results
print(f'Mean Angle Difference: JPL astrometric vs. JPL apparent')
print(f'Mean  : {diff_mean:5.0f} seconds ({(diff_mean/3600):0.3f} degrees)')
print(f'Median: {diff_median:5.0f} seconds ({(diff_median/3600):0.3f} degrees)')
print(f'Max   : {diff_max:5.0f} seconds ({(diff_max/3600):0.3f} degrees)')

**Conclusion**<br>
The difference between astrometric and apparent RA / DEC is really important!<br>
It's much more important than some of the other effects considered.<br>
It introduces errors on the order of **0.21 degrees / 743 arc seconds**<br>
We need to figure out the quotation basis of the ZTF data!<br>
Francisco from Alerce says he believes ZTF data "must be" astrometric.  Hopefully this is correct!